<a href="https://colab.research.google.com/github/rus4787/Sales-quota-algorithm-for-managers/blob/main/%D0%90%D0%9B%D0%93%D0%9E%D0%A0%D0%98%D0%A2%D0%9C%D0%AB_%D0%BA%D0%B2%D0%BE%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сложные алгоритмы

## Определение льготных интервалов продаж

Описание функции:
- Входные данные: threshold (float или int): Порог менеджера. Может содержать дополнительные символы (например, пробелы, запятые).
- Выходные данные:
    - основные_интервалы (list): Список основных интервалов продаж (каждый интервал — кортеж (min_value, max_value)).
    - квоты_основных_интервалов (dict): Словарь, где ключ — интервал, значение= квота для основного интервала.
    - льготные_интервалы (list): Список льготных интервалов продаж.
    - квоты_льготных_интервалов (dict): Словарь с квотами для льготных интервалов (по умолчанию квота 0).

Как работает функция:
- Инициализация интервалов и квот:
    - Интервалы продаж и их начальные квоты задаются в списке intervals и словаре initial_quotas.
- Обработка порога:
    - Удаление лишних символов из threshold.
    - Преобразование threshold в число.
    - Если threshold меньше 15000, устанавливается threshold = 15000.
    - Округление порога до целого числа.
- Определение основных и льготных интервалов:
    - Проходим по каждому интервалу продаж:
        - Если интервал пересекается с порогом, разбиваем его на льготный и основной.
        - Если интервал полностью ниже порога, он становится льготным.
        - Если интервал полностью выше порога, он остается основным.
- Настройка квот:
    - Для льготных интервалов квота устанавливается в 0.
    - Для основных интервалов квота берется из initial_quotas.
    - Стимулирующий интервал (следующий после интервала, содержащего порог) получает квоту, умноженную на 2.
- Возврат результатов: Функция возвращает кортеж с основными интервалами и их квотами, а также льготными интервалами и их квотами.

In [ ]:
def determine_sale_intervals(user_id, threshold):
    """
    Определяет основные и льготные интервалы продаж и соответствующие квоты для менеджера на основе его порога.

    Параметры:
    threshold (float или int): Порог менеджера.

    Возвращает:
    tuple:
        - main_intervals (list): Список основных интервалов продаж (каждый интервал - кортеж (min_value, max_value)).
        - main_interval_quotas (dict): Словарь с квотами для основных интервалов продаж.
        - preferential_intervals (list): Список льготных интервалов продаж.
        - preferential_interval_quotas (dict): Словарь с квотами для льготных интервалов продаж (по умолчанию квота == 0).
    """
   # Очистка и проверка порога
    try:
        threshold_value = float(str(threshold).replace(' ', '').replace(',', '').replace('.', ''))
    except ValueError:
        raise ValueError(f"Недопустимый ввод порога для user_id {user_id}.")

    if threshold_value < 15000:
        threshold_value = 15000

    threshold_value = int(threshold_value)  # Округление порога до целого числа

    main_intervals = []
    main_interval_quotas = {}
    preferential_intervals = []
    preferential_interval_quotas = {}

    stimulating_interval_assigned = False
    stimulating_interval = None

    intervals = [
        (5000, 10000),
        (10001, 15000),
        (15001, 20000),
        (20001, 25000),
        (25001, 30000),
        (30001, 35000),
        (35001, float('inf'))
    ]

    initial_quotas = {
        (5000, 10000): 0,
        (10001, 15000): 0,
        (15001, 20000): 5,
        (20001, 25000): 5,
        (25001, 30000): 5,
        (30001, 35000): 5,
        (35001, float('inf')): 99
    }

    for idx, interval in enumerate(intervals):
        min_value, max_value = interval

        # Проверяем, пересекается ли интервал с порогом
        if min_value < threshold_value <= max_value:
            # Разбиваем интервал на льготный и основной
            preferential_interval = (min_value, threshold_value - 1)
            main_interval = (threshold_value, max_value)

            # Добавляем льготный интервал
            preferential_intervals.append(preferential_interval)
            preferential_interval_quotas[preferential_interval] = 0

            # Добавляем основной интервал
            main_intervals.append(main_interval)
            main_interval_quotas[main_interval] = initial_quotas.get(interval, 5)

            # Назначаем следующий интервал как стимулирующий
            if idx + 1 < len(intervals):
                stimulating_interval = intervals[idx + 1]

        elif max_value <= threshold_value - 1:
            # Интервал полностью льготный
            preferential_intervals.append(interval)
            preferential_interval_quotas[interval] = 0

        elif min_value >= threshold_value:
            # Интервал полностью основной
            main_intervals.append(interval)
            quota = initial_quotas.get(interval, 5)

            # Увеличиваем квоту стимулирующего интервала
            if not stimulating_interval_assigned and stimulating_interval == interval:
                quota *= 2
                stimulating_interval_assigned = True

            main_interval_quotas[interval] = quota

        else:
            # Интервал выше порога и не требует действий
            main_intervals.append(interval)
            main_interval_quotas[interval] = initial_quotas.get(interval, 5)

    # Если стимулирующий интервал не был назначен (например, порог выше всех интервалов)
    if stimulating_interval and not stimulating_interval_assigned:
        main_interval_quotas[stimulating_interval] = initial_quotas.get(stimulating_interval, 5) * 2

    return {user_id: (main_intervals, main_interval_quotas, preferential_intervals, preferential_interval_quotas)}


# Пример использования:
# threshold = 22000
# user_id = 1
# manager_1 = determine_sale_intervals(user_id, threshold)

**Пояснение для программиста:**
- Основные интервалы продаж:
    - (22000, 25000) с квотой 5
    - (25001, 30000) с квотой 10 (это стимулирующий интервал с увеличенной квотой)
    - (30001, 35000) с квотой 5
    - (35001, inf) с квотой 99
- Льготные интервалы продаж:
    - (5000, 10000) с квотой 0
    - (10001, 15000) с квотой 0
    - (15001, 20000) с квотой 0
    - (20001, 21999) с квотой 0

Важно:
- Интервал (20001, 25000) был разделен на:
    - Льготный интервал (20001, 21999) с квотой 0
    - Основной интервал (22000, 25000) с квотой 5
- Стимулирующий интервал (25001, 30000) получил квоту, увеличенную в 2 раза, то есть 10 вместо 5.

Пояснения:
- Разделение интервала:
    - Интервал (20001, 25000) разбивается на:
    - Льготный интервал (20001, 21999) с квотой 0.
    - Основной интервал (22000, 25000) с квотой из initial_quotas.
- Стимулирующий интервал:
    - Следующий интервал после того, в котором находится порог (22000, 25000), — это (25001, 30000).
    - Его квота увеличивается в 2 раза: 5 * 2 = 10.

Примечания:
- Функция учитывает все требования, описанные в условии.
- При необходимости функция может быть расширена для обработки дополнительных условий или изменений в алгоритме.

## Расчет квот льготных интервалов

**Класс SaleProcessor**

In [ ]:
import threading
from queue import Queue

class SaleProcessor:
    def __init__(self):
        # Словарь менеджеров по user_id
        self.managers = {}
        # Очередь заявок
        self.queue = Queue()
        # Запуск воркеров
        for _ in range(5):  # Количество воркеров можно настроить
            threading.Thread(target=self.worker, daemon=True).start()
        # Вызов планирования очистки:
        self.schedule_clearing_processed_bids()

    def add_manager(self, manager):
        self.managers[manager.user_id] = manager

    def submit_sale(self, user_id, bid_id, sale_amount):
        # Помещаем заявку в очередь
        self.queue.put((user_id, bid_id, sale_amount))

    def worker(self):
        while True:
            user_id, bid_id, sale_amount = self.queue.get()
            try:
                manager = self.managers.get(user_id)
                if manager:
                    success = manager.process_sale(bid_id, sale_amount)
                    if success:
                        print(f"Сделка {bid_id} менеджера {user_id} успешно проведена.")
                    else:
                        print(f"Сделка {bid_id} менеджера {user_id} не может быть проведена.")
                else:
                    print(f"Менеджер с user_id {user_id} не найден.")
            finally:
                self.queue.task_done()

    # Методы для планирования и выполнения очистки:
    def schedule_clearing_processed_bids(self):
        threading.Timer(604800, self.clear_all_processed_bids).start()  # 604800 секунд = 1 неделя

    def clear_all_processed_bids(self):
        for manager in self.managers.values():
            manager.clear_processed_bids()
        # Повторно планируем следующую очистку
        self.schedule_clearing_processed_bids()


**class Manager**

In [ ]:
from threading import RLock

class Manager:
    """
    Класс для представления менеджера, его квот и методов обработки сделок.
    """

    def __init__(self, data_dict):
        """
        Инициализирует менеджера с заданными параметрами.

    Параметры:
        - data_dict (dict): Словарь данных, где ключи - user_id, значения - данные менеджера.
        """
        # Предполагаем, что data_dict имеет структуру {user_id: manager_data}
        # manager_data содержит ключи:
        # 'threshold', 'current_ssd', 'total_closed_deals', 'number_of_closed_deals',
        # 'main_intervals', 'main_interval_quotas', 'preferential_intervals', 'preferential_interval_quotas'

        if not data_dict:
            raise ValueError("Data dictionary is empty.")

        if len(data_dict) != 1:
            raise ValueError("Data dictionary should contain data for one manager.")

        self.user_id = next(iter(data_dict))
        manager_data = data_dict[self.user_id]

        self.threshold = manager_data.get('threshold', 15000)
        self.current_ssd = manager_data.get('current_ssd', 0.0)
        self.total_closed_deals = manager_data.get('total_closed_deals', 0.0)
        self.number_of_closed_deals = manager_data.get('number_of_closed_deals', 0)
        self.processed_bids = set()

        # Вычисляем зону открытия
        self.opening_zone = manager_data.get('opening_zone')
        if self.opening_zone is None:
            # Calculate opening zone if not provided
            opening_zone_dict = calculate_opening_zone(self.user_id, self.threshold)
            self.opening_zone = opening_zone_dict[self.user_id]

        # Инициализируем интервалы и квоты
        self.main_intervals = manager_data.get('main_intervals')
        self.main_interval_quotas = manager_data.get('main_interval_quotas')
        self.preferential_intervals = manager_data.get('preferential_intervals')
        self.preferential_interval_quotas = manager_data.get('preferential_interval_quotas')

        if not all([self.main_intervals, self.main_interval_quotas,
                    self.preferential_intervals, self.preferential_interval_quotas]):
            # If intervals and quotas are not provided, calculate them
            intervals_dict = determine_sale_intervals(self.user_id, self.threshold)
            intervals = intervals_dict[self.user_id]
            self.main_intervals = intervals[0]
            self.main_interval_quotas = intervals[1]
            self.preferential_intervals = intervals[2]
            self.preferential_interval_quotas = intervals[3]

        # Блокировка для обеспечения потокобезопасности
        self.lock = RLock()

    def clear_processed_bids(self):
        with self.lock:
            self.processed_bids.clear()

    def check_interval(self, sale_amount):
        """
        Определяет тип и ключ интервала для заданной суммы сделки.

        Параметры:
        - sale_amount (float): Сумма сделки.

        Возвращает:
        - (interval_type, interval_key): Кортеж с типом и ключом интервала.
        - None: Если интервал не найден.
        """
        # Проверяем основные интервалы
        for interval in self.main_intervals:
            min_value, max_value = interval
            if min_value <= sale_amount <= max_value or (max_value == float('inf') and sale_amount >= min_value):
                return "main", interval

        # Проверяем льготные интервалы
        for interval in self.preferential_intervals:
            min_value, max_value = interval
            if min_value <= sale_amount <= max_value:
                return "preferential", interval

        return None

    def check_sale(self, bid_id, sale_amount):
        """
        Проверяет возможность проведения сделки без изменения состояния.

        Параметры:
        - bid_id (int): Идентификатор сделки.
        - sale_amount (float): Сумма сделки.

        Возвращает:
        - can_proceed (bool): True, если сделка может быть проведена, False в противном случае.
        """
        # Проверяем, не был ли bid_id уже обработан
        if bid_id in self.processed_bids:
            return False

        # Определяем интервал сделки
        interval_info = self.check_interval(sale_amount)
        if not interval_info:
            return False

        interval_type, interval_key = interval_info

        # Проверяем наличие квоты в интервале
        if interval_type == "main":
            quota = self.main_interval_quotas.get(interval_key, 0)
        else:
            quota = self.preferential_interval_quotas.get(interval_key, 0)

        if quota <= 0:
            # Нет доступной квоты
            return False

        # Рассчитываем новый ССД
        new_total_closed_deals = self.total_closed_deals + sale_amount
        new_number_of_closed_deals = self.number_of_closed_deals + 1
        new_current_ssd = new_total_closed_deals / new_number_of_closed_deals

        # Проверяем условия
        if new_current_ssd < self.threshold:
            return False

        if interval_type == "preferential" and new_current_ssd < self.opening_zone:
            return False

        return True

    def process_sale(self, bid_id, sale_amount):
        """
        Обрабатывает проведение сделки, обновляя квоты и состояние менеджера.

        Параметры:
        - bid_id (int): Идентификатор сделки.
        - sale_amount (float): Сумма сделки.

        Возвращает:
        - success (bool): True, если сделка успешно проведена, False в противном случае.
        """
        with self.lock:
            # Проверяем, не был ли bid_id уже обработан
            if bid_id in self.processed_bids:
                return False
            can_proceed = self.check_sale(bid_id, sale_amount)
            if not can_proceed:
                return False
            # Добавляем bid_id в обработанные
            self.processed_bids.add(bid_id)

            # Обновляем квоты
            interval_info = self.check_interval(sale_amount)
            if interval_info:
                interval_type, interval_key = interval_info
                if interval_type == "main":
                    self.main_interval_quotas[interval_key] -= 1
                else:
                    self.preferential_interval_quotas[interval_key] -= 1
            else:
                return False

            # Обновляем показатели
            self.total_closed_deals += sale_amount
            self.number_of_closed_deals += 1
            self.current_ssd = self.total_closed_deals / self.number_of_closed_deals

            # Проверяем, нужно ли закрыть льготные квоты
            if self.current_ssd < self.opening_zone:
                for key in self.preferential_interval_quotas:
                    self.preferential_interval_quotas[key] = 0

            return True

# Простые алгоритмы

## Расчет текущего ССД менеджера в любой момент времени

Описание функции:
- Входные данные: closed_deals (list): Список сумм закрытых заявок менеджера. Элементы могут быть числовыми значениями (int, float) или строками, содержащими числа и дополнительные символы (например, пробелы, запятые, точки, текст).
- Выходные данные: float: Текущий ССД менеджера. Если после обработки нет корректных заявок, функция возвращает 0.0.

Как работает функция:
- Проверяет, что список closed_deals не пустой.
- Инициализирует пустой список cleaned_deals для хранения очищенных сумм заявок.
- Для каждого элемента в closed_deals:
    - Преобразует элемент в строку.
    - Удаляет все символы, кроме цифр, точки и запятой.
    - Заменяет запятые на точки для корректного преобразования в число с плавающей точкой.
    - Попытаться преобразовать очищенную строку в float.
        - Если успешно, добавляет сумму в cleaned_deals.
        - сли возникает ошибка, игнорирует этот элемент.
- Если после обработки нет корректных заявок (список cleaned_deals пустой), возвращает 0.0.
- Вычисляет сумму всех корректных заявок (total_sum).
- Определяет количество корректных заявок (n).
- Рассчитывает текущий ССД как total_sum / n.
- Возвращает полученное значение current_ssd.

In [ ]:
import re

def calculate_current_ssd(user_id, closed_deals):
    """
    Вычисляет текущий ССД менеджера на основе списка закрытых заявок.

    Параметры:
    closed_deals (list): Список сумм закрытых заявок менеджера. Элементы могут быть числовыми значениями или строками, содержащими числа и лишние символы.

    Возвращает:
    float: Текущий ССД менеджера. Если нет корректных заявок, возвращает 0.0.

    Примечания:
    - Функция очищает входные данные, удаляя некорректные символы.
    - Игнорирует заявки, которые не удается преобразовать в число.
    """
    if not closed_deals:
        return {user_id: 0.0}

    # Список для очищенных сумм заявок
    cleaned_deals = []

    for deal in closed_deals:
        # Преобразование заявки в строку
        deal_str = str(deal)

        # Удаление всех нецифровых символов, кроме точки и запятой
        deal_cleaned = re.sub(r'[^\d.,]', '', deal_str)

        # Замена запятой на точку для корректного преобразования в float
        deal_cleaned = deal_cleaned.replace(',', '.')

        # Попытка преобразовать очищенную строку в число
        try:
            amount = float(deal_cleaned)
            cleaned_deals.append(amount)
        except ValueError:
            continue  # Игнорируем некорректные значения

    # Проверка наличия корректных данных
    if not cleaned_deals:
        return {user_id: 0.0}

    # Расчет суммы всех закрытых заявок
    total_sum = sum(cleaned_deals)
    # Количество закрытых заявок
    n = len(cleaned_deals)
    # Расчет текущего ССД
    current_ssd = total_sum / n

    return {user_id: current_ssd}

# Пример использования:
# closed_deals = [10000, '15 000', '20,000', '25000.00', 'не число', 30000]
# user_id = 1
# current_ssd = calculate_current_ssd(user_id, closed_deals)
# print(current_ssd)  # Вывод: средний ССД


Пояснение:
- Функция обрабатывает различные форматы записи сумм, включая:
    - Разделители тысяч (пробелы, запятые).
    - Десятичные разделители (точки, запятые).
- Некорректные значения, которые не удается преобразовать в число (например, 'не число'), игнорируются.
- В результате вычисляется среднее значение сумм корректных закрытых заявок.

Примечания:
- Функция может быть модифицирована для учета специфики формата ввода (например, если известно, что десятичный разделитель всегда запятая).
- При необходимости можно добавить обработку исключений или логирование некорректных значений для последующего анализа.

## Алгоритм "Определение зоны открытия для каждого менеджера"

Описание:
- Входные данные: threshold (float или int): порог менеджера. Должен быть не меньше 15,000.
- Выходные данные: словарь с ключом user_id и значением — результатом работы функции: int: зона открытия, округленная до ближайшего целого числа.

Как работает функция:
- Проверяет, меньше ли порог 20,000.
    - Если да, зона открытия равна порог плюс 2,000.
    - Если нет, зона открытия равна порог, умноженный на 1.1.
- Округляет полученное значение зоны открытия до ближайшего целого числа.
- Возвращает зону открытия.

In [ ]:
import re

def calculate_opening_zone(user_id, threshold):
    """
    Вычисляет зону открытия для менеджера на основе его порога.
    Параметры:
    threshold (str, float или int): Порог менеджера. Может содержать лишние символы.

    Возвращает:
    int: Зона открытия, округленная до ближайшего целого числа.

    Примечания:
    - Если threshold меньше 15 000, устанавливается threshold = 15 000.
    - Функция очищает входные данные, удаляя недопустимые символы.
    """
    # Преобразование threshold в строку
    threshold_str = str(threshold)

    # Удаление всех нецифровых символов
    threshold_cleaned = re.sub(r'\D', '', threshold_str)

    # Проверка, что после очистки осталось значение
    if not threshold_cleaned:
        raise ValueError(f"Недопустимый ввод порога для user_id {user_id}.")

    # Преобразование в целое число
    threshold_value = int(threshold_cleaned)

    # Проверка минимального значения
    if threshold_value < 15000:
        threshold_value = 15000

    # Вычисление зоны открытия
    if threshold_value < 20000:
        opening_zone = threshold_value + 2000
    else:
        opening_zone = threshold_value * 1.1

    # Округление до ближайшего целого числа
    opening_zone = round(opening_zone)

    return {user_id: opening_zone}

# Пример использования:
# user_id = 1
# opening_zone = calculate_opening_zone(user_id,"20.200")
# print(opening_zone)  # Вывод: 22200


# Полный цикл кода

In [ ]:
import re
import threading
from queue import Queue
from threading import RLock

# Функция для вычисления зоны открытия
def calculate_opening_zone(user_id, threshold):
    """
    Вычисляет зону открытия для менеджера на основе его порога.
    Возвращает словарь {user_id: opening_zone}
    """
    # Преобразование threshold в строку
    threshold_str = str(threshold)

    # Удаление всех нецифровых символов
    threshold_cleaned = re.sub(r'\D', '', threshold_str)

    # Проверка, что после очистки осталось значение
    if not threshold_cleaned:
        raise ValueError(f"Недопустимый ввод порога для user_id {user_id}.")

    # Преобразование в целое число
    threshold_value = int(threshold_cleaned)

    # Проверка минимального значения
    if threshold_value < 15000:
        threshold_value = 15000

    # Вычисление зоны открытия
    if threshold_value < 20000:
        opening_zone = threshold_value + 2000
    else:
        opening_zone = threshold_value * 1.1

    # Округление до ближайшего целого числа
    opening_zone = round(opening_zone)

    return {user_id: opening_zone}

# Функция для определения интервалов продаж
def determine_sale_intervals(user_id, threshold):
    """
    Определяет основные и льготные интервалы продаж и соответствующие квоты для менеджера.
    Возвращает словарь {user_id: (main_intervals, main_interval_quotas, preferential_intervals, preferential_interval_quotas)}
    """
    # Очистка и проверка порога
    try:
        threshold_value = float(str(threshold).replace(' ', '').replace(',', '').replace('.', ''))
    except ValueError:
        raise ValueError(f"Недопустимый ввод порога для user_id {user_id}.")

    if threshold_value < 15000:
        threshold_value = 15000

    threshold_value = int(threshold_value)  # Округление порога до целого числа

    main_intervals = []
    main_interval_quotas = {}
    preferential_intervals = []
    preferential_interval_quotas = {}

    stimulating_interval_assigned = False
    stimulating_interval = None

    intervals = [
        (5000, 10000),
        (10001, 15000),
        (15001, 20000),
        (20001, 25000),
        (25001, 30000),
        (30001, 35000),
        (35001, float('inf'))
    ]

    initial_quotas = {
        (5000, 10000): 0,
        (10001, 15000): 0,
        (15001, 20000): 5,
        (20001, 25000): 5,
        (25001, 30000): 5,
        (30001, 35000): 5,
        (35001, float('inf')): 99
    }

    for idx, interval in enumerate(intervals):
        min_value, max_value = interval

        # Проверяем, пересекается ли интервал с порогом
        if min_value < threshold_value <= max_value:
            # Разбиваем интервал на льготный и основной
            preferential_interval = (min_value, threshold_value - 1)
            main_interval = (threshold_value, max_value)

            # Добавляем льготный интервал
            preferential_intervals.append(preferential_interval)
            preferential_interval_quotas[preferential_interval] = 0

            # Добавляем основной интервал
            main_intervals.append(main_interval)
            main_interval_quotas[main_interval] = initial_quotas.get(interval, 5)

            # Назначаем следующий интервал как стимулирующий
            if idx + 1 < len(intervals):
                stimulating_interval = intervals[idx + 1]

        elif max_value <= threshold_value - 1:
            # Интервал полностью льготный
            preferential_intervals.append(interval)
            preferential_interval_quotas[interval] = 0

        elif min_value >= threshold_value:
            # Интервал полностью основной
            main_intervals.append(interval)
            quota = initial_quotas.get(interval, 5)

            # Увеличиваем квоту стимулирующего интервала
            if not stimulating_interval_assigned and stimulating_interval == interval:
                quota *= 2
                stimulating_interval_assigned = True

            main_interval_quotas[interval] = quota

        else:
            # Интервал выше порога и не требует действий
            main_intervals.append(interval)
            main_interval_quotas[interval] = initial_quotas.get(interval, 5)

    # Если стимулирующий интервал не был назначен (например, порог выше всех интервалов)
    if stimulating_interval and not stimulating_interval_assigned:
        main_interval_quotas[stimulating_interval] = initial_quotas.get(stimulating_interval, 5) * 2

    return {user_id: (main_intervals, main_interval_quotas, preferential_intervals, preferential_interval_quotas)}

# Функция для расчета текущего ССД
def calculate_current_ssd(user_id, closed_deals):
    """
    Вычисляет текущий ССД менеджера.
    Возвращает словарь {user_id: current_ssd}
    """
    if not closed_deals:
        return {user_id: 0.0}

    # Список для очищенных сумм заявок
    cleaned_deals = []

    for deal in closed_deals:
        # Преобразование заявки в строку
        deal_str = str(deal)

        # Удаление всех нецифровых символов, кроме точки и запятой
        deal_cleaned = re.sub(r'[^\d.,]', '', deal_str)

        # Замена запятой на точку для корректного преобразования в float
        deal_cleaned = deal_cleaned.replace(',', '.')

        # Попытка преобразовать очищенную строку в число
        try:
            amount = float(deal_cleaned)
            cleaned_deals.append(amount)
        except ValueError:
            continue  # Игнорируем некорректные значения

    # Проверка наличия корректных данных
    if not cleaned_deals:
        return {user_id: 0.0}

    # Расчет суммы всех закрытых заявок
    total_sum = sum(cleaned_deals)
    # Количество закрытых заявок
    n = len(cleaned_deals)
    # Расчет текущего ССД
    current_ssd = total_sum / n

    return {user_id: current_ssd}

# Функция для получения очищенных сумм сделок (понадобится для вычисления total_closed_deals и number_of_closed_deals)
def get_cleaned_deals(closed_deals):
    cleaned_deals = []

    for deal in closed_deals:
        # Преобразование заявки в строку
        deal_str = str(deal)

        # Удаление всех нецифровых символов, кроме точки и запятой
        deal_cleaned = re.sub(r'[^\d.,]', '', deal_str)

        # Замена запятой на точку для корректного преобразования в float
        deal_cleaned = deal_cleaned.replace(',', '.')

        # Попытка преобразовать очищенную строку в число
        try:
            amount = float(deal_cleaned)
            cleaned_deals.append(amount)
        except ValueError:
            continue  # Игнорируем некорректные значения
    return cleaned_deals

# Класс менеджера
class Manager:
    """
    Класс для представления менеджера, его квот и методов обработки сделок.
    """
    def __init__(self, data_dict):
        """
        Инициализирует менеджера с данными из data_dict.

        Параметры:
        - data_dict (dict): Словарь данных, где ключи - user_id, значения - данные менеджера.
        """
        # Предполагаем, что data_dict имеет структуру {user_id: manager_data}
        # manager_data содержит ключи:
        # 'threshold', 'current_ssd', 'total_closed_deals', 'number_of_closed_deals',
        # 'main_intervals', 'main_interval_quotas', 'preferential_intervals', 'preferential_interval_quotas'

        if not data_dict:
            raise ValueError("Data dictionary is empty.")

        if len(data_dict) != 1:
            raise ValueError("Data dictionary should contain data for one manager.")

        self.user_id = next(iter(data_dict))
        manager_data = data_dict[self.user_id]

        self.threshold = manager_data.get('threshold', 15000)
        self.current_ssd = manager_data.get('current_ssd', 0.0)
        self.total_closed_deals = manager_data.get('total_closed_deals', 0.0)
        self.number_of_closed_deals = manager_data.get('number_of_closed_deals', 0)
        self.processed_bids = set()

        # Вычисляем зону открытия
        self.opening_zone = manager_data.get('opening_zone')
        if self.opening_zone is None:
            # Calculate opening zone if not provided
            opening_zone_dict = calculate_opening_zone(self.user_id, self.threshold)
            self.opening_zone = opening_zone_dict[self.user_id]

        # Инициализируем интервалы и квоты
        self.main_intervals = manager_data.get('main_intervals')
        self.main_interval_quotas = manager_data.get('main_interval_quotas')
        self.preferential_intervals = manager_data.get('preferential_intervals')
        self.preferential_interval_quotas = manager_data.get('preferential_interval_quotas')

        if not all([self.main_intervals, self.main_interval_quotas,
                    self.preferential_intervals, self.preferential_interval_quotas]):
            # If intervals and quotas are not provided, calculate them
            intervals_dict = determine_sale_intervals(self.user_id, self.threshold)
            intervals = intervals_dict[self.user_id]
            self.main_intervals = intervals[0]
            self.main_interval_quotas = intervals[1]
            self.preferential_intervals = intervals[2]
            self.preferential_interval_quotas = intervals[3]

        # Блокировка для обеспечения потокобезопасности
        self.lock = RLock()

    def clear_processed_bids(self):
        with self.lock:
            self.processed_bids.clear()

    def check_interval(self, sale_amount):
        """
        Определяет тип и ключ интервала для заданной суммы сделки.

        Параметры:
        - sale_amount (float): Сумма сделки.

        Возвращает:
        - (interval_type, interval_key): Кортеж с типом и ключом интервала.
        - None: Если интервал не найден.
        """
        # Проверяем основные интервалы
        for interval in self.main_intervals:
            min_value, max_value = interval
            if min_value <= sale_amount <= max_value or (max_value == float('inf') and sale_amount >= min_value):
                return "main", interval

        # Проверяем льготные интервалы
        for interval in self.preferential_intervals:
            min_value, max_value = interval
            if min_value <= sale_amount <= max_value:
                return "preferential", interval

        return None

    def check_sale(self, bid_id, sale_amount):
        """
        Проверяет возможность проведения сделки без изменения состояния.

        Параметры:
        - bid_id (int): Идентификатор сделки.
        - sale_amount (float): Сумма сделки.

        Возвращает:
        - can_proceed (bool): True, если сделка может быть проведена, False в противном случае.
        """
        # Проверяем, не был ли bid_id уже обработан
        if bid_id in self.processed_bids:
            return False

        # Определяем интервал сделки
        interval_info = self.check_interval(sale_amount)
        if not interval_info:
            return False

        interval_type, interval_key = interval_info

        # Проверяем наличие квоты в интервале
        if interval_type == "main":
            quota = self.main_interval_quotas.get(interval_key, 0)
        else:
            quota = self.preferential_interval_quotas.get(interval_key, 0)

        if quota <= 0:
            # Нет доступной квоты
            return False

        # Рассчитываем новый ССД
        new_total_closed_deals = self.total_closed_deals + sale_amount
        new_number_of_closed_deals = self.number_of_closed_deals + 1
        new_current_ssd = new_total_closed_deals / new_number_of_closed_deals

        # Проверяем условия
        if new_current_ssd < self.threshold:
            return False

        if interval_type == "preferential" and new_current_ssd < self.opening_zone:
            return False

        return True

    def process_sale(self, bid_id, sale_amount):
        """
        Обрабатывает проведение сделки, обновляя квоты и состояние менеджера.

        Параметры:
        - bid_id (int): Идентификатор сделки.
        - sale_amount (float): Сумма сделки.

        Возвращает:
        - success (bool): True, если сделка успешно проведена, False в противном случае.
        """
        with self.lock:
            # Проверяем, не был ли bid_id уже обработан
            if bid_id in self.processed_bids:
                return False
            can_proceed = self.check_sale(bid_id, sale_amount)
            if not can_proceed:
                return False
            # Добавляем bid_id в обработанные
            self.processed_bids.add(bid_id)

            # Обновляем квоты
            interval_info = self.check_interval(sale_amount)
            if interval_info:
                interval_type, interval_key = interval_info
                if interval_type == "main":
                    self.main_interval_quotas[interval_key] -= 1
                else:
                    self.preferential_interval_quotas[interval_key] -= 1
            else:
                return False

            # Обновляем показатели
            self.total_closed_deals += sale_amount
            self.number_of_closed_deals += 1
            self.current_ssd = self.total_closed_deals / self.number_of_closed_deals

            # Проверяем, нужно ли закрыть льготные квоты
            if self.current_ssd < self.opening_zone:
                for key in self.preferential_interval_quotas:
                    self.preferential_interval_quotas[key] = 0

            return True

# Класс для обработки заявок
class SaleProcessor:
    def __init__(self):
        # Словарь менеджеров по user_id
        self.managers = {}
        # Очередь заявок
        self.queue = Queue()
        # Запуск воркеров
        for _ in range(5):  # Количество воркеров можно настроить
            threading.Thread(target=self.worker, daemon=True).start()
        # Вызов планирования очистки:
        self.schedule_clearing_processed_bids()

    def add_manager(self, manager):
        self.managers[manager.user_id] = manager

    def submit_sale(self, user_id, bid_id, sale_amount):
        # Помещаем заявку в очередь
        self.queue.put((user_id, bid_id, sale_amount))

    def worker(self):
        while True:
            user_id, bid_id, sale_amount = self.queue.get()
            try:
                manager = self.managers.get(user_id)
                if manager:
                    success = manager.process_sale(bid_id, sale_amount)
                    if success:
                        print(f"Сделка {bid_id} менеджера {user_id} успешно проведена.")
                    else:
                        print(f"Сделка {bid_id} менеджера {user_id} не может быть проведена.")
                else:
                    print(f"Менеджер с user_id {user_id} не найден.")
            finally:
                self.queue.task_done()

    # Методы для планирования и выполнения очистки:
    def schedule_clearing_processed_bids(self):
        threading.Timer(604800, self.clear_all_processed_bids).start()  # 604800 секунд = 1 неделя

    def clear_all_processed_bids(self):
        for manager in self.managers.values():
            manager.clear_processed_bids()
        # Повторно планируем следующую очистку
        self.schedule_clearing_processed_bids()


In [ ]:
# --- Основной код ---

# Данные менеджера
user_id = 1
threshold = 22000
closed_deals = [10000, '15 000', '20,000', '25000.00', 'не число', 30000]

# Вычисляем текущий ССД
current_ssd_dict = calculate_current_ssd(user_id, closed_deals)
current_ssd = current_ssd_dict[user_id]

# Получаем очищенные сделки
cleaned_deals = get_cleaned_deals(closed_deals)
total_closed_deals = sum(cleaned_deals)
number_of_closed_deals = len(cleaned_deals)

# Вычисляем зону открытия
opening_zone_dict = calculate_opening_zone(user_id, threshold)
opening_zone = opening_zone_dict[user_id]

# Определяем интервалы продаж и квоты
intervals_dict = determine_sale_intervals(user_id, threshold)
(main_intervals, main_interval_quotas, preferential_intervals, preferential_interval_quotas) = intervals_dict[user_id]

# Подготавливаем данные менеджера
manager_data = {
    user_id: {
        'threshold': threshold,
        'current_ssd': current_ssd,
        'total_closed_deals': total_closed_deals,
        'number_of_closed_deals': number_of_closed_deals,
        'opening_zone': opening_zone,
        'main_intervals': main_intervals,
        'main_interval_quotas': main_interval_quotas,
        'preferential_intervals': preferential_intervals,
        'preferential_interval_quotas': preferential_interval_quotas
    }
}

# Создаем экземпляр менеджера
manager = Manager(manager_data)

# Создаем экземпляр SaleProcessor и добавляем менеджера
processor = SaleProcessor()
processor.add_manager(manager)

# Подаем сделку на обработку
bid_id = 101
sale_amount = 10000
processor.submit_sale(user_id=user_id, bid_id=bid_id, sale_amount=sale_amount)

# Ждем обработки всех заявок
processor.queue.join()